# Advanced Querying Mongo

Importing libraries and setting up connection

In [21]:
# pip install pymongo

In [55]:
from pymongo import MongoClient
import pandas as pd
pd.options.display.max_columns = None

In [20]:
cursor = MongoClient()   # no esta escrita la string de conexion

In [11]:
cursor.list_database_names()

['admin', 'companies', 'config', 'local']

In [23]:
db = cursor.companies   # Acceso a la base de datos

In [19]:
db.list_collection_names()

['colecion']

In [31]:
colec = db.colecion # Acceso a la colección?

In [37]:
res = list(colec.find())
df = pd.DataFrame(res)

In [56]:
df.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,video_embeds,screenshots,external_links,partners,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,2005.0,10.0,17.0,1.0,"wiki, seattle, elowitz, media-industry, media-...",,info@wetpaint.com,206.859.6300,Technology Platform Company,2007-05-25 06:51:27,Sun Dec 08 07:15:44 UTC 2013,<p>Wetpaint is a technology platform company t...,"{'available_sizes': [[[150, 75], 'assets/image...","[{'name': 'Wikison Wetpaint', 'permalink': 'we...","[{'is_past': False, 'title': 'Co-Founder and V...","[{'competitor': {'name': 'Wikia', 'permalink':...",[],$39.8M,"[{'id': 888, 'round_code': 'a', 'source_url': ...",[],"{'price_amount': 30000000, 'price_currency_cod...",[],"[{'description': '', 'address1': '710 - 2nd Av...","[{'id': 5869, 'description': 'Wetpaint named i...",[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,1996.0,NaN,NaN,2.0,,Zoho ManageEngine,pr@adventnet.com,925-924-9500,Server Management Software,2007-05-25 19:24:22,Wed Oct 31 18:26:09 UTC 2012,"<p>AdventNet is now <a href=""/company/zoho-man...","{'available_sizes': [[[150, 55], 'assets/image...",[],"[{'is_past': True, 'title': 'CEO and Co-Founde...",[],"[{'title': 'DHFH', 'is_past': True, 'provider'...",$0,[],[],None,[],"[{'description': 'Headquarters', 'address1': '...",[],[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,2005.0,9.0,15.0,3.0,"zoho, officesuite, spreadsheet, writer, projec...",,info@zohocorp.com,1-888-204-3539,Online Business Apps Suite,Fri May 25 19:30:28 UTC 2007,Wed Oct 30 00:07:05 UTC 2013,"<p>Zoho offers a suite of Business, Collaborat...","{'available_sizes': [[[150, 55], 'assets/image...","[{'name': 'Zoho Office Suite', 'permalink': 'z...","[{'is_past': False, 'title': 'CEO and Founder'...","[{'competitor': {'name': 'Empressr', 'permalin...",[],$0,[],[],None,[],"[{'description': 'Headquarters', 'address1': '...","[{'id': 388, 'description': 'Zoho Reaches 2 Mi...","[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],NaN,NaN,NaN,NaN
3,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,2004.0,10.0,11.0,NaN,"community, social, news, bookmark, digg, techn...",,feedback@digg.com,(415) 436-9638,user driven social content website,Fri May 25 20:03:23 UTC 2007,Tue Nov 05 21:35:47 UTC 2013,<p>Digg is a user driven social content websit...,"{'available_sizes': [[[150, 150], 'assets/imag...","[{'name': 'Digg', 'permalink': 'digg'}]","[{'is_past': False, 'title': 'CEO', 'person': ...","[{'competitor': {'name': 'Reddit', 'permalink'...","[{'title': 'Public Relations', 'is_past': True...",$45M,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...",[],"{'price_amount': 500000, 'price_currency_code'...","[{'price_amount': None, 'price_currency_code':...","[{'description': None, 'address1': '135 Missis...","[{'id': 9588, 'description': 'Another Digg Exe...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'ava

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [48]:
query = {'name': 'Babelgum'}

select = {'_id':False, 'name': 1}

list(colec.find(query,select))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [52]:
query = {'number_of_employees': {'$gte':5000}}

select = {'_id':False, 'name': 1, 'number_of_employees':1}

list(colec.find(query,select).sort('number_of_employees', -1).limit(20))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [63]:
query = {'$and': [{'founded_year': {'$lte': 2005}},
                  {'founded_year': {'$gte': 2000}}
                 ]}

select = {'_id':False, 'name': 1, 'founded_year':1}

#list(colec.find(query,select))
list(colec.find(query,select).limit(20))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'SmugMug', 'founded_year': 2002},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'Skype', 'founded_year': 2003}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [78]:
query = {'$and': [{'founded_year': {'$lte': 2010}},
                  {'ipo.valuation_amount': {'$gte': 100000000}}
                 ]}

select = {'_id': False, 'name': 1, 'ipo.valuation_amount': 1}

list(colec.find(query, select).limit(20))

[{'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Yelp', 'ipo': {'valuation_amount': 1300000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Brightcove', 'ipo': {'valuation_amount': 290000000}},
 {'name': 'KIT digital', 'ipo': {'valuation_amount': 235000000}},
 {'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'name': 'OpenTable', 'ipo': {'valuation_amount': 1050000000}},
 {'name': 'ChannelAdvisor', 'ipo': {'valuation_amount': 287000000}},
 {'name': 'Jive Software', 'ipo': {'valuation_amount': 1000000000}},
 {'name': 'Zillow', 'ipo': {'valuation_amount': 2550000000.0}},
 {'name': 'Wix', 'ipo': {'valuation_amount': 750000000}},
 {'name': 'Shutterfly', 'ipo': {'valuation_amount': 350000000}},
 {'name': 'TripAdvisor', 'ipo': {'valuation_amount': 3273770000.0}},
 {'name': 'Salesforce

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [86]:
query = {'$and': [{'founded_year': {'$lte': 2010}},
                  {'number_of_employees': {'$lte': 1000}},
                  {'number_of_employees': {'$ne': 0}}
                 ]}

select = {'_id': False, 'name': 1, 'number_of_employees': 1}

list(colec.find(query, select).sort('number_of_employees',1).limit(10))

[{'name': 'FeVote', 'number_of_employees': 1},
 {'name': 'OurStage', 'number_of_employees': 1},
 {'name': 'eBurl', 'number_of_employees': 1},
 {'name': 'Localeze', 'number_of_employees': 1},
 {'name': 'Contrastream', 'number_of_employees': 1},
 {'name': 'Bizak', 'number_of_employees': 1},
 {'name': 'Blogonize', 'number_of_employees': 1},
 {'name': 'Entrecard', 'number_of_employees': 1},
 {'name': 'eGenerations', 'number_of_employees': 1},
 {'name': 'Swivel', 'number_of_employees': 1}]

### 6. All the companies that don't include the `partners` field.

In [99]:
query = {'partners': {'$size': 0}}

select = {'_id': False, 'name':1, 'partners': 1}

list(colec.find(query, select).limit(10))

[{'name': 'Wetpaint', 'partners': []},
 {'name': 'AdventNet', 'partners': []},
 {'name': 'Zoho', 'partners': []},
 {'name': 'Digg', 'partners': []},
 {'name': 'Facebook', 'partners': []},
 {'name': 'Omnidrive', 'partners': []},
 {'name': 'Postini', 'partners': []},
 {'name': 'Geni', 'partners': []},
 {'name': 'Flektor', 'partners': []},
 {'name': 'Fox Interactive Media', 'partners': []}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [108]:
query = {'category_code': None}

select = {'_id': False, 'name':1, 'category_code': 1}

list(colec.find(query, select).limit(10))

[{'name': 'Collective', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None},
 {'name': 'Level9 Media', 'category_code': None},
 {'name': 'VidKing', 'category_code': None},
 {'name': 'Drigg', 'category_code': None},
 {'name': 'SpaceTime', 'category_code': None},
 {'name': 'Touch Clarity', 'category_code': None},
 {'name': 'MMDAYS', 'category_code': None},
 {'name': 'Inside Group', 'category_code': None}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [106]:
query = {'$and':[{'number_of_employees': {'$gte': 100}},
                {'number_of_employees': {'$lte': 1000}}
                 ]}

select = {'_id': False, 'name':1, 'number_of_employees': 1}

list(colec.find(query, select).limit(10))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [134]:
query = {'ipo.valuation_amount': {'$gte': 0}}

select = {'_id': False, 'name': 1, 'ipo.valuation_amount': 1}

list(colec.find(query, select).sort('ipo.valuation_amount',-1))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000.0}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000.0}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000.0}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000.0}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000.0}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000.0}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000.0}},
 {'name': 'Baidu', 'ipo': {'valuation_amount': 4000000000.0}},
 {'name': 'TripAdvisor', 'ipo': {'valuation_amount': 3273770000.0}},
 {'name': 'HomeAway', 'ipo': {'valuation_amount': 3000000000.0}},
 {'name': 'Zillow', 'ipo': {'valuation_amount': 2550000000.0}},
 {'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [135]:
query = {}

select = {'_id': False, 'name':1, 'number_of_employees': 1}

list(colec.find(query, select).sort('number_of_employees',-1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [146]:
query = {'founded_month': {'$gt':6}}

select = {'_id': False, 'name':1, 'founded_month': 1}

list(colec.find(query, select).limit(1000))

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'eBay', 'founded_month': 9},
 {'name': 'Joost', 'founded_month': 10},
 {'name': 'Plaxo', 'founded_month': 11},
 {'name': 'Powerset', 'founded_month': 10},
 {'name': 'Kyte', 'founded_month': 12},
 {'name': 'Thoof', 'founded_month': 12},
 {'name': 'Jingle Networks', 'founded_month': 9},
 {'name': 'LifeLock', 'founded_month': 8},
 {'name': 'Wesabe', 'founded_month': 12},
 {'name': 'SmugMug', 'founded_month': 11},
 {'name': 'Google', 'founded_month': 9},
 {'name': 'Skype', 'founded_month': 8},
 {'name': 'Pando Networks', 'founded_month': 7},
 {'name': 'Ikan', 'founded_month': 12},
 {'name': 'delicious', 'founded_month': 9},
 {'name': 'AllPeers', 'founded_month': 8},
 {'name': 'Wize', 'founded_month': 8},
 {'name': 'SellABand', 'founded_month': 8},
 {'name': 'iContact', 'founded_month': 7},
 {'name': 'MeeVee', 'foun

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [157]:
query = {'$and': [{'founded_year': {'$lt': 2000}},
                  {'acquisition.price_amount': {'$gt': 10_000_000}}
                 ]}

select = {'_id': False, 'name':1, 'founded': 1, 'acquisition.price_amount': 1}

list(colec.find(query, select).limit(10))

[{'name': 'Postini', 'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep', 'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar', 'acquisition': {'price_amount': 25000000}},
 {'name': 'PayPal', 'acquisition': {'price_amount': 1500000000}},
 {'name': 'Snapfish', 'acquisition': {'price_amount': 300000000}},
 {'name': 'Neopets', 'acquisition': {'price_amount': 160000000}},
 {'name': 'Sun Microsystems', 'acquisition': {'price_amount': 7400000000.0}},
 {'name': 'Zappos', 'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba', 'acquisition': {'price_amount': 1000000000}},
 {'name': 'Sabre', 'acquisition': {'price_amount': 4300000000.0}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [160]:
query = {'acquisition.acquired_year': {'$gt': 2010}}

select = {'_id': False, 'name':1, 'acquisition': 1}

list(colec.find(query, select).sort('acquisition.price_amount', -1).limit(10))

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name':

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [167]:
select = {'_id': False, 'name':1, 'founded_year': 1}

list(colec.find(query, select).sort('founded_year', -1).limit(10))

[{'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [172]:
query = {'$and':[{'founded_day': {'$lte':7}},
                 {'founded_day': {'$gt':0}}
                ]}

select = {'_id':False, 'name':True, 'founded_day':True}

list(colec.find(query, select).sort('acquisition.price_amount', -1).limit(10))

[{'name': 'Netscape', 'founded_day': 4},
 {'name': 'PayPal', 'founded_day': 1},
 {'name': 'Zappos', 'founded_day': 1},
 {'name': 'Alibaba', 'founded_day': 1},
 {'name': 'Postini', 'founded_day': 2},
 {'name': 'Danger', 'founded_day': 1},
 {'name': 'Clearwell Systems', 'founded_day': 6},
 {'name': 'PrimeSense', 'founded_day': 1},
 {'name': 'Amobee', 'founded_day': 1},
 {'name': 'Vitrue', 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [176]:
query = {'$and':[{'category_code': 'web'},
                 {'number_of_employees': {'$gte':4000}}
                ]}

select = {'_id':False, 'name':True, 'number_of_employees':True}

list(colec.find(query, select).sort('number_of_employees', 1).limit(10))

[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group', 'number_of_employees': 10000},
 {'name': 'Groupon', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'number_of_employees': 13600},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Experian', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [180]:
query = {'$and':[{'acquisition.price_amount': {'$gte':10_000_000}},
                 {'acquisition.price_currency_code': 'EUR'}
                ]}

select = {'_id':False, 'name':True, 
            'acquisition.price_currency_code':True,
            'acquisition.price_amount':True}

list(colec.find(query, select).sort('acquisition.price_amount',-1))

[{'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'Tuenti Technologies',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'BioMed Central',
  'acquisition': {'price_amount': 43400000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [184]:
query = {'acquisition.acquired_month': {'$lte':3}}

select = {'_id':False, 'name':True, 'acquisition.acquired_month':True}

list(colec.find(query, select).limit(10))

[{'name': 'Kyte', 'acquisition': {'acquired_month': 1}},
 {'name': 'NetRatings', 'acquisition': {'acquired_month': 2}},
 {'name': 'blogTV', 'acquisition': {'acquired_month': 3}},
 {'name': 'Livestream', 'acquisition': {'acquired_month': 1}},
 {'name': 'iContact', 'acquisition': {'acquired_month': 2}},
 {'name': 'Coghead', 'acquisition': {'acquired_month': 2}},
 {'name': 'Dailymotion', 'acquisition': {'acquired_month': 2}},
 {'name': 'Netvibes', 'acquisition': {'acquired_month': 2}},
 {'name': 'Flickr', 'acquisition': {'acquired_month': 3}},
 {'name': 'BabyCenter', 'acquisition': {'acquired_month': 3}}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [188]:
query = {'$and':[{'founded_year': {'$gte':2000}},
                 {'founded_year': {'$lte':2010}},
                 {'acquisition.acquired_year': {'$gte':2011}}
                ]}

select = {'_id':False, 'name':True, 'founded_year':True, 'acquisition.acquired_year':True}

list(colec.find(query, select).limit(10))

[{'name': 'Wetpaint',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Digg',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Geni',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Kyte',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2011}},
 {'name': 'Jingle Networks',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2011}},
 {'name': 'blogTV',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'delicious',
  'founded_year': 2003,
  'acquisition': {'acquired_year': 2011}},
 {'name': 'Revision3',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'iContact',
  'founded_year': 2003,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Mashery',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2013}}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [196]:
query = {'$and': [
        {'deadpooled_year': {'$ne': None}},  # Nos aseguramos de que el valor existe
        {'$expr': # iniciamos la expresión de comparación
         { '$gt': # comparamos con 3. Al fijarse en los corchetes concuerdan
          [{ '$subtract': # restamos la diferencia entre dead y founded
            ['$deadpooled_year', '$founded_year'] }, 3]}}]}


select = {'_id':False, 'name':True, 'deadpooled_year':True}

list(colec.find(query, select).limit(10))

[{'name': 'Babelgum', 'deadpooled_year': 2013},
 {'name': 'Thoof', 'deadpooled_year': 2013},
 {'name': 'Wesabe', 'deadpooled_year': 2010},
 {'name': 'Stickam', 'deadpooled_year': 2013},
 {'name': 'AllPeers', 'deadpooled_year': 2008},
 {'name': 'EQO', 'deadpooled_year': 2012},
 {'name': 'AllofMP3', 'deadpooled_year': 2007},
 {'name': 'SellABand', 'deadpooled_year': 2010},
 {'name': 'Zlio', 'deadpooled_year': 2011},
 {'name': 'Jaiku', 'deadpooled_year': 2012}]